In [ ]:
#Import standard packages for model training

import tensorflow as T
from tensorflow import *
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras import layers
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras.losses import *
import numpy as np
import pandas as pd
import sklearn.metrics as SK
from sklearn.metrics import *
import matplotlib.pyplot as plt
import os 
import io
from utils import utils
from utils.GPU import *
commons = utils.Commons()
ts_helper = utils.TS_Helper()
shap_helper = utils.Shap_Helper()
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [5]:
#Import train, validation and test sets

training = './data/Classification/T.cruzi/scaffold_split/train_fold_0.csv'
validation = './data/Classification/T.cruzi/scaffold_split/valid_fold_1.csv'
test = './data/Classification/T.cruzi/scaffold_split/test_fold_2.csv'
TASK_START = 2
NUM_TASKS = 1

train_dataset,y_train = commons.load_dataset(training,TASK_START,NUM_TASKS)
val_dataset,y_val = commons.load_dataset(validation,TASK_START,NUM_TASKS)
test_dataset,y_test = commons.load_dataset(test,TASK_START,NUM_TASKS)
train_dataset.head()

Loaded dataset ./data/Classification/T.cruzi/scaffold_split/train_fold_0.csv with shape: (4914, 3)
Loaded dataset ./data/Classification/T.cruzi/scaffold_split/valid_fold_1.csv with shape: (615, 3)
Loaded dataset ./data/Classification/T.cruzi/scaffold_split/test_fold_2.csv with shape: (614, 3)


,ID,SMILES,T.cruzi
0,596,[H]N(C(=O)C1=C([H])C([H])=C([H])S1)C1=C(OC([H]...,1
1,5514,[H]N1C(=O)N(C([H])=C(F)C1=O)C1([H])OC([H])(C([...,0
2,916,[H]C1=C([H])N(C([H])=N1)C1=C([H])C([H])=C(C([H...,1
3,5413,[H]SC1=NN=C(N1C1([H])C([H])([H])C([H])([H])C([...,0
4,547,[H]C1=NC([H])=C(C([H])=C1[H])N(C1=C([H])C([H])...,1


In [8]:
#Import atom and bond featurizers

from utils.graphs import *

X_train = graphs_from_smiles(train_dataset.iloc[:,NUM_TASKS].values)
print(train_dataset.iloc[:,NUM_TASKS].values)
X_test = graphs_from_smiles(test_dataset.iloc[:,NUM_TASKS].values)

X_val = graphs_from_smiles(val_dataset.iloc[:,NUM_TASKS].values)

molecule = molecule_from_smiles(train_dataset.iloc[8].SMILES)
graph = graph_from_molecule(molecule)
print("Graph (including self-loops):")
print("\tatom features\t", graph[0].shape)
print("\tbond features\t", graph[1].shape)
print("\tpair indices\t", graph[2].shape)

array(['[H]N(C(=O)C1=C([H])C([H])=C([H])S1)C1=C(OC([H])([H])[H])C([H])=C([H])C(=C1[H])C1=NC2=C(O1)C([H])=C([H])C([H])=N2',
       '[H]N1C(=O)N(C([H])=C(F)C1=O)C1([H])OC([H])(C([H])([H])OC(=O)C([H])([H])[H])C([H])(OC(=O)C([H])([H])[H])C1([H])OC(=O)C([H])([H])[H]',
       '[H]C1=C([H])N(C([H])=N1)C1=C([H])C([H])=C(C([H])=C1[H])C([H])([H])N1C([H])([H])C([H])([H])N(C([H])([H])C1([H])[H])S(=O)(=O)C1=C(C([H])=C([H])C([H])=C1[H])C(F)(F)F',
       ...,
       '[H]N(C(=O)C([H])=C([H])C1=C(Cl)C([H])=C([H])C([H])=C1F)C1=NC([H])=C([H])C([H])=N1',
       '[H]C1=C([H])C([H])=C(C([H])=C1[H])N1C2=C(C([H])=C(OC(=O)C3=C([H])C(OC([H])([H])[H])=C(OC([H])([H])[H])C(OC([H])([H])[H])=C3[H])C([H])=C2[H])C(C(=O)OC([H])([H])C([H])([H])[H])=C1C([H])([H])[H]',
       '[H]N(C(=O)C([H])(N1C([H])=C([H])C([H])=C([H])C1=O)C(=O)C1=C([H])C([H])=C(F)C([H])=C1[H])C([H])([H])C1=NC([H])=C([H])C([H])=C1[H]'],
      dtype=object)
Graph (including self-loops):
	atom features	 (42, 29)
	bond features	 (128, 7)
	pair indices	 (1

In [ ]:
# Import parameters for batch, MPNN, loss and scorers

batch_size =48
dense_units = 50

from utils.batch import *
from utils.MPNN import *
from utils.utils import *


# parameters for train network

def MPNNModel(atom_dim,
    bond_dim,
    batch_size=batch_size,
    message_units=48,
    message_steps=8,
    num_attention_heads=16,
    dense_units=dense_units):

    atom_features = layers.Input((atom_dim), dtype="float32", name="atom_features")
    bond_features = layers.Input((bond_dim), dtype="float32", name="bond_features")
    pair_indices = layers.Input((2), dtype="int32", name="pair_indices")
    molecule_indicator = layers.Input((), dtype="int32", name="molecule_indicator")

    x = MessagePassing(message_units, message_steps)([atom_features, bond_features, pair_indices])
    x = TransformerEncoderReadout(num_attention_heads, message_units, dense_units, batch_size)([x, molecule_indicator])
    x = layers.Dense(10, activation="sigmoid", kernel_regularizer=T.keras.regularizers.L2(0.05))(x)

    
    model = keras.Model(inputs=[atom_features, bond_features, pair_indices, molecule_indicator],
        outputs=[x])
    return model
    


optimizer = RMSprop(0.1, momentum=0.99)
lr_metric = ts_helper.get_lr_metric(optimizer)   

model = MPNNModel(atom_dim=X_train[0][0][0].shape[0], bond_dim=X_train[1][0][0].shape[0])
model.compile(loss = ts_helper.classification_loss(ts_helper.BinaryCrossentropy), metrics=[lr_metric])   
model.summary()
T.keras.utils.plot_model(model)

In [ ]:
# Early stopping parameters

train_dataset = MPNNDataset(X_train, y_train, batch_size)
val_dataset = MPNNDataset(X_val, y_val, batch_size)
test_dataset = MPNNDataset(X_test, y_test, batch_size)

callbacks_list = [
    ReduceLROnPlateau(monitor='loss', factor=0.5, patience=10, min_lr=0.000000001, verbose=1, mode='auto',cooldown=0),
    #ModelCheckpoint(filepath="./models/fine_tuned_model2.tf", monitor='val_loss', save_best_only=True, verbose=1, mode='auto'),
    EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, mode='min', verbose=1)]

# parameters for train network

epochs = 2000

history = model.fit(train_dataset,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=val_dataset,                    
                    callbacks=(callbacks_list))


In [ ]:
#Plot model history

ts_helper.plot_history(history)

In [ ]:
#Statistical characteristics of tasks
THRESHOLD = 0.5
ts_helper.get_modelStats(model,X_train,X_test,X_val,y_train,y_val,y_test,NUM_TASKS,THRESHOLD)